# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f1f04859460>
├── 'a' --> tensor([[-1.7194,  0.6632,  2.0242],
│                   [-0.9415, -0.1848,  1.6534]])
└── 'x' --> <FastTreeValue 0x7f1f048591c0>
    └── 'c' --> tensor([[ 0.6495,  1.2800,  1.0320, -0.4778],
                        [-0.7640, -0.7569, -0.7914, -0.2461],
                        [-0.4614, -2.2736,  1.8402, -1.2693]])

In [4]:
t.a

tensor([[-1.7194,  0.6632,  2.0242],
        [-0.9415, -0.1848,  1.6534]])

In [5]:
%timeit t.a

75.9 ns ± 0.0459 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f1f04859460>
├── 'a' --> tensor([[-0.4120,  0.7070,  0.2828],
│                   [-0.8855, -0.2902, -0.0261]])
└── 'x' --> <FastTreeValue 0x7f1f048591c0>
    └── 'c' --> tensor([[ 0.6495,  1.2800,  1.0320, -0.4778],
                        [-0.7640, -0.7569, -0.7914, -0.2461],
                        [-0.4614, -2.2736,  1.8402, -1.2693]])

In [7]:
%timeit t.a = new_value

80.4 ns ± 0.118 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-1.7194,  0.6632,  2.0242],
               [-0.9415, -0.1848,  1.6534]]),
    x: Batch(
           c: tensor([[ 0.6495,  1.2800,  1.0320, -0.4778],
                      [-0.7640, -0.7569, -0.7914, -0.2461],
                      [-0.4614, -2.2736,  1.8402, -1.2693]]),
       ),
)

In [10]:
b.a

tensor([[-1.7194,  0.6632,  2.0242],
        [-0.9415, -0.1848,  1.6534]])

In [11]:
%timeit b.a

65 ns ± 0.14 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.9475,  0.3726, -1.4400],
               [-2.6636,  0.5933,  0.7950]]),
    x: Batch(
           c: tensor([[ 0.6495,  1.2800,  1.0320, -0.4778],
                      [-0.7640, -0.7569, -0.7914, -0.2461],
                      [-0.4614, -2.2736,  1.8402, -1.2693]]),
       ),
)

In [13]:
%timeit b.a = new_value

575 ns ± 1.77 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

887 ns ± 0.491 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.2 µs ± 30.8 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

143 µs ± 1.28 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

145 µs ± 878 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f1e2a42e700>
├── 'a' --> tensor([[[-1.7194,  0.6632,  2.0242],
│                    [-0.9415, -0.1848,  1.6534]],
│           
│                   [[-1.7194,  0.6632,  2.0242],
│                    [-0.9415, -0.1848,  1.6534]],
│           
│                   [[-1.7194,  0.6632,  2.0242],
│                    [-0.9415, -0.1848,  1.6534]],
│           
│                   [[-1.7194,  0.6632,  2.0242],
│                    [-0.9415, -0.1848,  1.6534]],
│           
│                   [[-1.7194,  0.6632,  2.0242],
│                    [-0.9415, -0.1848,  1.6534]],
│           
│                   [[-1.7194,  0.6632,  2.0242],
│                    [-0.9415, -0.1848,  1.6534]],
│           
│                   [[-1.7194,  0.6632,  2.0242],
│                    [-0.9415, -0.1848,  1.6534]],
│           
│                   [[-1.7194,  0.6632,  2.0242],
│                    [-0.9415, -0.1848,  1.6534]]])
└── 'x' --> <FastTreeValue 0x7f1e3202ffd0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

33.4 µs ± 24.8 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f1e2a0a8fd0>
├── 'a' --> tensor([[-1.7194,  0.6632,  2.0242],
│                   [-0.9415, -0.1848,  1.6534],
│                   [-1.7194,  0.6632,  2.0242],
│                   [-0.9415, -0.1848,  1.6534],
│                   [-1.7194,  0.6632,  2.0242],
│                   [-0.9415, -0.1848,  1.6534],
│                   [-1.7194,  0.6632,  2.0242],
│                   [-0.9415, -0.1848,  1.6534],
│                   [-1.7194,  0.6632,  2.0242],
│                   [-0.9415, -0.1848,  1.6534],
│                   [-1.7194,  0.6632,  2.0242],
│                   [-0.9415, -0.1848,  1.6534],
│                   [-1.7194,  0.6632,  2.0242],
│                   [-0.9415, -0.1848,  1.6534],
│                   [-1.7194,  0.6632,  2.0242],
│                   [-0.9415, -0.1848,  1.6534]])
└── 'x' --> <FastTreeValue 0x7f1f048595b0>
    └── 'c' --> tensor([[ 0.6495,  1.2800,  1.0320, -0.4778],
                        [-0.7640, -0.7569, -0.7914, -0.2461],
                 

In [23]:
%timeit t_cat(trees)

31.1 µs ± 64.6 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

68.2 µs ± 97.4 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[-1.7194,  0.6632,  2.0242],
                [-0.9415, -0.1848,  1.6534]],
       
               [[-1.7194,  0.6632,  2.0242],
                [-0.9415, -0.1848,  1.6534]],
       
               [[-1.7194,  0.6632,  2.0242],
                [-0.9415, -0.1848,  1.6534]],
       
               [[-1.7194,  0.6632,  2.0242],
                [-0.9415, -0.1848,  1.6534]],
       
               [[-1.7194,  0.6632,  2.0242],
                [-0.9415, -0.1848,  1.6534]],
       
               [[-1.7194,  0.6632,  2.0242],
                [-0.9415, -0.1848,  1.6534]],
       
               [[-1.7194,  0.6632,  2.0242],
                [-0.9415, -0.1848,  1.6534]],
       
               [[-1.7194,  0.6632,  2.0242],
                [-0.9415, -0.1848,  1.6534]]]),
    x: Batch(
           c: tensor([[[ 0.6495,  1.2800,  1.0320, -0.4778],
                       [-0.7640, -0.7569, -0.7914, -0.2461],
                       [-0.4614, -2.2736,  1.8402, -1.2693]],
         

In [26]:
%timeit Batch.stack(batches)

84.1 µs ± 195 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[-1.7194,  0.6632,  2.0242],
               [-0.9415, -0.1848,  1.6534],
               [-1.7194,  0.6632,  2.0242],
               [-0.9415, -0.1848,  1.6534],
               [-1.7194,  0.6632,  2.0242],
               [-0.9415, -0.1848,  1.6534],
               [-1.7194,  0.6632,  2.0242],
               [-0.9415, -0.1848,  1.6534],
               [-1.7194,  0.6632,  2.0242],
               [-0.9415, -0.1848,  1.6534],
               [-1.7194,  0.6632,  2.0242],
               [-0.9415, -0.1848,  1.6534],
               [-1.7194,  0.6632,  2.0242],
               [-0.9415, -0.1848,  1.6534],
               [-1.7194,  0.6632,  2.0242],
               [-0.9415, -0.1848,  1.6534]]),
    x: Batch(
           c: tensor([[ 0.6495,  1.2800,  1.0320, -0.4778],
                      [-0.7640, -0.7569, -0.7914, -0.2461],
                      [-0.4614, -2.2736,  1.8402, -1.2693],
                      [ 0.6495,  1.2800,  1.0320, -0.4778],
                      [-0.7640, -

In [28]:
%timeit Batch.cat(batches)

154 µs ± 142 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

355 µs ± 3.92 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
